In [ ]:
!pip install requests beautifulsoup4 lxml selenium webdriver-manager pandas

In [ ]:
!apt-get update
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f # Install dependencies
print("Google Chrome has been installed.")

Get:1 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://cli.github.com/packages stable InRelease
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:10 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,200 B]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,025 B in 2s (1,727 B/s)
Reading package lists... Done
W: Skipping a

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://myanimelist.net/topanime.php")
time.sleep(3)

data = []

# Find all ranking list rows
ranking_rows = driver.find_elements(By.CSS_SELECTOR, "tr.ranking-list")

for row in ranking_rows:
    title = ""
    score = ""
    status_type = ""
    episodes = ""
    aired_schedule = ""
    genre = "" # Genre is not directly available on this page without navigating to individual anime pages

    try:
        title_element = row.find_element(By.CSS_SELECTOR, "h3.anime_ranking_h3 > a.hoverinfo_trigger")
        title = title_element.text.strip()
    except:
        pass # Handle cases where title might not be found

    try:
        score_element = row.find_element(By.CSS_SELECTOR, "div.js-top-ranking-score-col > span.score-label")
        score = score_element.text.strip()
    except:
        pass # Handle cases where score might not be found

    try:
        info_element = row.find_element(By.CSS_SELECTOR, "div.information.di-ib.mt4")
        info_text = info_element.text.strip()

        if info_text:
            lines = info_text.split('\n')

            if len(lines) > 0:
                first_line = lines[0].strip()
                # Extract status type (e.g., TV, Movie, OVA, ONA, Special)
                status_match = re.search(r"^(TV|Movie|OVA|ONA|Special)", first_line)
                if status_match:
                    status_type = status_match.group(0)

                # Extract episodes (e.g., (28 eps), (1 eps), (? eps))
                episodes_match = re.search(r"\((\d+)\s*eps\)", first_line) # Only capture numbers
                if episodes_match:
                    episodes = int(episodes_match.group(1))
                elif "? eps" in first_line:
                    episodes = None # Represent '?' as None or NaN


            if len(lines) > 1:
                aired_schedule = lines[1].strip()
    except:
        pass # Handle cases where info might not be found or parsing fails

    if title:
        data.append({
            "title": title,
            "score": score,
            "status": status_type,
            "episodes": episodes,
            "aired_schedule": aired_schedule,
            "genre": genre
        })

driver.quit()

df = pd.DataFrame(data)
df

,title,score,status,episodes,aired_schedule,genre
0,Sousou no Frieren,9.29,TV,28.0,Sep 2023 - Mar 2024,
1,Chainsaw Man Movie: Reze-hen,9.18,Movie,1.0,Sep 2025 - Sep 2025,
2,Fullmetal Alchemist: Brotherhood,9.10,TV,64.0,Apr 2009 - Jul 2010,
3,Steins;Gate,9.07,TV,24.0,Apr 2011 - Sep 2011,
4,Kingdom 6th Season,9.06,TV,NaN,Oct 2025 -,
5,Shingeki no Kyojin Season 3 Part 2,9.05,TV,10.0,Apr 2019 - Jul 2019,
6,Gintama: The Final,9.05,Movie,1.0,Jan 2021 - Jan 2021,
7,Gintama°,9.05,TV,51.0,Apr 2015 - Mar 2016,
8,Hunter x Hunter (2011),9.03,TV,148.0,Oct 2011 - Sep 2014,
9,Ginga Eiyuu Densetsu,9.02,OVA,110.0,Jan 1988 - Mar 1997,


In [ ]:
df.to_csv("scraping_result.csv", index=False)
print("File scraping_result.csv berhasil dibuat")

File scraping_result.csv berhasil dibuat


# Task
Extract the URLs for individual anime pages from the main listing, then visit each of these pages to scrape the genre information. Integrate the scraped genre data with the existing anime data (title, score, status, aired schedule), and convert the 'episodes' column to a numeric type, handling missing values as `NaN`. Finally, save the updated dataset, including the new genre information and cleaned episode numbers, to `scraping_result.csv`.

## Extract Individual Anime URLs

### Subtask:
Modify the current scraping script to extract the direct URL to each anime's detailed page from the main list. These URLs will be stored to allow for further navigation.


**Reasoning**:
I need to modify the existing scraping script to extract the anime URL from the 'title_element' and add it to the data dictionary. This involves finding the anchor tag within the title element and getting its 'href' attribute.



In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time
import re

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://myanimelist.net/topanime.php")
time.sleep(3)

data = []

# Find all ranking list rows
ranking_rows = driver.find_elements(By.CSS_SELECTOR, "tr.ranking-list")

for row in ranking_rows:
    title = ""
    score = ""
    status_type = ""
    episodes = ""
    aired_schedule = ""
    genre = "" # Genre is not directly available on this page without navigating to individual anime pages
    anime_url = "" # Initialize anime_url

    try:
        title_element = row.find_element(By.CSS_SELECTOR, "h3.anime_ranking_h3 > a.hoverinfo_trigger")
        title = title_element.text.strip()
        anime_url = title_element.get_attribute('href') # Extract the URL
    except:
        pass # Handle cases where title or url might not be found

    try:
        score_element = row.find_element(By.CSS_SELECTOR, "div.js-top-ranking-score-col > span.score-label")
        score = score_element.text.strip()
    except:
        pass # Handle cases where score might not be found

    try:
        info_element = row.find_element(By.CSS_SELECTOR, "div.information.di-ib.mt4")
        info_text = info_element.text.strip()

        if info_text:
            lines = info_text.split('\n')

            if len(lines) > 0:
                first_line = lines[0].strip()
                # Extract status type (e.g., TV, Movie, OVA, ONA, Special)
                status_match = re.search(r"^(TV|Movie|OVA|ONA|Special)", first_line)
                if status_match:
                    status_type = status_match.group(0)

                # Extract episodes (e.g., (28 eps), (1 eps), (? eps))
                episodes_match = re.search(r"\((\d+)\s*eps\)", first_line) # Only capture numbers
                if episodes_match:
                    episodes = int(episodes_match.group(1))
                elif "? eps" in first_line:
                    episodes = None # Represent '?' as None or NaN


            if len(lines) > 1:
                aired_schedule = lines[1].strip()
    except:
        pass # Handle cases where info might not be found or parsing fails

    if title:
        data.append({
            "title": title,
            "score": score,
            "status": status_type,
            "episodes": episodes,
            "aired_schedule": aired_schedule,
            "genre": genre,
            "url": anime_url # Add the extracted URL here
        })

driver.quit()

df = pd.DataFrame(data)
df

,title,score,status,episodes,aired_schedule,genre,url
0,Sousou no Frieren,9.29,TV,28.0,Sep 2023 - Mar 2024,,https://myanimelist.net/anime/52991/Sousou_no_...
1,Chainsaw Man Movie: Reze-hen,9.18,Movie,1.0,Sep 2025 - Sep 2025,,https://myanimelist.net/anime/57555/Chainsaw_M...
2,Fullmetal Alchemist: Brotherhood,9.10,TV,64.0,Apr 2009 - Jul 2010,,https://myanimelist.net/anime/5114/Fullmetal_A...
3,Steins;Gate,9.07,TV,24.0,Apr 2011 - Sep 2011,,https://myanimelist.net/anime/9253/Steins_Gate
4,Kingdom 6th Season,9.06,TV,NaN,Oct 2025 -,,https://myanimelist.net/anime/61517/Kingdom_6t...
5,Shingeki no Kyojin Season 3 Part 2,9.05,TV,10.0,Apr 2019 - Jul 2019,,https://myanimelist.net/anime/38524/Shingeki_n...
6,Gintama: The Final,9.05,Movie,1.0,Jan 2021 - Jan 2021,,https://myanimelist.net/anime/39486/Gintama__T...
7,Gintama°,9.05,TV,51.0,Apr 2015 - Mar 2016,,https://myanimelist.net/anime/28977/Gintama%C2%B0
8,Hunter x Hunter (2011),9.03,TV,148.0,Oct 2011 - Sep 2014,,https://myanimelist.net/anime/11061/Hunter_x_H...
9,Ginga Eiyuu Densetsu,9.02,OVA,110.0,Jan 1988 - Mar 1997,,https://myanimelist.net/anime/820/Ginga_Eiyuu_...


**Reasoning**:
Now that the URLs have been extracted, the next step is to iterate through each URL, visit the individual anime pages, and scrape the genre information. This will involve using Selenium to navigate to each URL and locate the genre elements on those pages. After scraping the genres, the `episodes` column needs to be converted to a numeric type, and the final DataFrame will be saved to a CSV file.



In [ ]:
import numpy as np

# Re-initialize the WebDriver for the new scraping pass
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Iterate through each row of the DataFrame to visit individual anime pages and scrape genre
for index, row in df.iterrows():
    anime_url = row['url']
    if anime_url:
        try:
            driver.get(anime_url)
            time.sleep(2) # Give the page some time to load

            # Find the genre information on the individual page
            # Genres are usually listed in a div with a specific class or property
            genre_elements = driver.find_elements(By.CSS_SELECTOR, "div.spaceit_pad > span.dark_text")
            genres = []
            for element in genre_elements:
                if "Genres:" in element.text:
                    # The genres are typically sibling <a> tags of the 'Genres:' span
                    # Or sometimes a direct text content of a parent/sibling element
                    # A more robust way might be to look for specific tags following 'Genres:'
                    parent_div = element.find_element(By.XPATH, "..")
                    genre_links = parent_div.find_elements(By.CSS_SELECTOR, "a")
                    genres = [link.text.strip() for link in genre_links]
                    break
            df.at[index, 'genre'] = ", ".join(genres)

        except Exception as e:
            print(f"Error scraping genre for {row['title']}: {e}")
            df.at[index, 'genre'] = ""

driver.quit()

# Convert 'episodes' column to numeric, coercing errors to NaN
df['episodes'] = pd.to_numeric(df['episodes'], errors='coerce')

# Save the updated DataFrame to a CSV file
df.to_csv("scraping_result.csv", index=False)
print("Updated scraping_result.csv with genre information and cleaned episodes.")
print(df.head())


Error scraping genre for Gintama': HTTPConnectionPool(host='localhost', port=39439): Read timed out. (read timeout=120)
Error scraping genre for Gintama': Enchousen: HTTPConnectionPool(host='localhost', port=39439): Read timed out. (read timeout=120)
Error scraping genre for One Piece Fan Letter: HTTPConnectionPool(host='localhost', port=39439): Read timed out. (read timeout=120)
Error scraping genre for Bleach: Sennen Kessen-hen: HTTPConnectionPool(host='localhost', port=39439): Read timed out. (read timeout=120)


KeyboardInterrupt: 